In [98]:
from pymavlink import mavutil

import pandas as pd

import geopandas as gpd
from shapely.geometry import Point, LineString

In [99]:
path_log = 'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - EQUIPO AVIACION UTEA/Trichogramma/log_vuelo_2025/logs04'
path_res = 'C:/Documents/Ingenio Azucarero Guabira S.A/UTEA - SEMANAL - EQUIPO AVIACION UTEA/Trichogramma/log_vuelo_2025/logs04_shp'

In [100]:
contenido = os.listdir(path_log)
len(contenido)

5

In [104]:
contenido

['2024-06-27 14-21-22.tlog',
 '2024-06-27 14-33-51.tlog',
 '2024-06-27 16-41-20.tlog',
 '2024-06-27 18-39-33.tlog',
 '2024-06-27 19-06-35.tlog']

In [106]:
def leer_logs(lista_logs):
    list_geo = []
    list_name = []
    for i in lista_logs:
        log = path_log + '/' + i
        mlog = mavutil.mavlink_connection(log)
        lats = []
        lons = []
        while True:
            msg = mlog.recv_match()
            if not msg:
                break
            # Ejemplo de procesamiento de mensaje
            if msg.get_type() == 'GLOBAL_POSITION_INT':
                lat = msg.lat / 1e7  # Latitud en grados
                lon = msg.lon / 1e7  # Longitud en grados
                lats.append(lat)
                lons.append(lon)

        coors = {'lat':lats, 'lon':lons}
        df = pd.DataFrame(coors)
        
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
        line = LineString(geometry)
        list_geo.append(line)
        list_name.append(i)
    #dict_lines = {'geos':list_geo, 'names':list_name}
    #gdf_line = gpd.GeoDataFrame(dict_lines, crs="EPSG:4326")
    #gdf_line_utm = gdf_line.to_crs(epsg=32720)
    return (list_geo, list_name)

In [107]:
l = leer_logs(contenido)

In [108]:
l

([<LINESTRING (-63.183 -17.243, -63.183 -17.243, -63.183 -17.243, -63.183 -17....>,
  <LINESTRING (-63.192 -17.241, -63.192 -17.241, -63.192 -17.241, -63.192 -17....>,
  <LINESTRING (-63.191 -17.241, -63.191 -17.241, -63.191 -17.241, -63.191 -17....>,
  <LINESTRING (-63.139 -17.262, -63.139 -17.262, -63.139 -17.262, -63.139 -17....>,
  <LINESTRING (-63.139 -17.262, -63.139 -17.262, -63.139 -17.262, -63.139 -17....>],
 ['2024-06-27 14-21-22.tlog',
  '2024-06-27 14-33-51.tlog',
  '2024-06-27 16-41-20.tlog',
  '2024-06-27 18-39-33.tlog',
  '2024-06-27 19-06-35.tlog'])

In [83]:
log = path_log + '/' + '2024-06-27 14-33-51.tlog'

In [84]:
mlog = mavutil.mavlink_connection(log)

In [85]:
lats = []
lons = []

In [86]:
while True:
    msg = mlog.recv_match()
    if not msg:
        break
    # Ejemplo de procesamiento de mensaje
    if msg.get_type() == 'GLOBAL_POSITION_INT':
        lat = msg.lat / 1e7  # Latitud en grados
        lon = msg.lon / 1e7  # Longitud en grados
        alt = msg.alt / 1e3  # Altitud en metros
        
        lats.append(lat)
        lons.append(lon)
        #print(f"Latitud: {lat}, Longitud: {lon}, Altitud: {alt}")

In [87]:
coors = {'lat':lats, 'lon':lons}

In [88]:
df = pd.DataFrame(coors)

In [89]:
df

,lat,lon
0,-17.240544,-63.191583
1,-17.240544,-63.191583
2,-17.240544,-63.191583
3,-17.240544,-63.191583
4,-17.240543,-63.191583
...,...,...
1281,-17.240540,-63.191579
1282,-17.240540,-63.191579
1283,-17.240539,-63.191579
1284,-17.240538,-63.191578


In [90]:
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry)

# Establecer el sistema de coordenadas original (WGS84)
gdf.set_crs(epsg=4326, inplace=True)
# Reproyectar a UTM zona 20S
gdf_utm = gdf.to_crs(epsg=32720)

line = LineString(geometry)
gdf_line = gpd.GeoDataFrame(geometry=[line], crs="EPSG:4326")
gdf_line_utm = gdf_line.to_crs(epsg=32720)

In [95]:
gdf_line_utm['name']='frgrgr'

In [96]:
gdf_line_utm

,geometry,name
0,"LINESTRING (479633.490 8093824.325, 479633.479...",frgrgr


In [97]:
output_path = path_res + '/output_line_shapefile.shp'
gdf_line_utm.to_file(output_path)